In [ ]:
# OpenCVを使った画像処理

In [ ]:
import cv2

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import urllib.request as req
url = "http://uta.pw/shodou/img/28/214.png"
req.urlretrieve(url,"test.png")


# OpenCVで読み込む
import cv2
img = cv2.imread("test.png")
print(img)

In [ ]:
import cv2
img = cv2.imread("test.png")
plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
cv2.imwrite("output.png",img)
plt.show()

In [ ]:
# 画像のリサイズ

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("test.png")

#画像の読み込み
im2 = cv2.resize(img,(600,300))

cv2.imwrite("output-resize.png", im2)

# 画像を表示
plt.imshow(cv2.cvtColor(im2,cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
#画像の切り取り

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("test.png")

#画像の一部をきりとる
im2 = img[150:450,150:450]

#画像のリサイズ
im2 = cv2.resize(im2,(400,400))

cv2.imwrite("cut-resize.png", im2)

# 画像を表示
plt.imshow(cv2.cvtColor(im2,cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
#顔検出

In [ ]:
import cv2
import matplotlib.pyplot as plt

# カスケードファイルを指定して検出器を作成
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

#画像を読み込んでグレイスケールにする
img = cv2.imread("girl.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 顔認識を実行
face_list = cascade.detectMultiScale(img_gray, minSize=(150,150))

# 結果を確認
if len(face_list) == 0:
    print("失敗")
    quit()

#認識した部分に印をつける
for (x,y,w,h) in face_list:
    print("顔の座標=",x,y,w,h)
    red = (0,0,255)
    cv2.rectangle(img, (x,y), (x+w,y+h) ,red, thickness=10)

#画像を出力
cv2.imwrite("face-detect.png", img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
# 顔にモザイクをかける

In [ ]:
def mosaic(img,rect,size):
    (x1,y1,x2,y2) = rect
    w = x2 - x1
    h = y2 - y1
    i_rect = img[y1:y2, x1:x2]
    
    i_small = cv2.resize(i_rect, (size,size))
    i_mos = cv2.resize(i_small, (w,h), interpolation=cv2.INTER_AREA)
    
    img2 = img.copy()
    img2[y1:y2, x1:x2] = i_mos
    return img2

In [ ]:
import cv2
import matplotlib.pyplot as plt
#from mosaic import mosaic as mosaic

# カスケードファイルを指定して検出器を作成
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

#画像を読み込んでグレイスケールにする
img = cv2.imread("girl.jpg")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 顔検出を実行
face_list = cascade.detectMultiScale(img_gray, minSize=(150,150))

# 結果を確認
if len(face_list) == 0:
    print("失敗")
    quit()

#認識した部分にモザイクをかける
for (x,y,w,h) in face_list:
    img = mosaic(img,(x,y,x+w,y+h),10)
    
#画像を出力
cv2.imwrite("face-mosaic.png", img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
#注意: OpenCVの顔検出は横顔や傾きに弱いらしい

In [ ]:
# 動画解析

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
while True:
    # カメラの画像を読み込む
    _,frame = cap.read()
    # 画像を縮小表示する
    frame = cv2.resize(frame, (500,300))
    #ウィンドウ画像を出力
    cv2.imshow('OpeCV Web Camera', frame)
    # ESCかEnterキーが押されたらループを抜ける
    k = cv2.waitKey(1) # 1msec
    if k == 27 or k == 13: break
        
cap.release() # カメラを解放
cv2.destroyAllWindows() # ウィンドウを破棄


In [ ]:
# 画像に動きがあった部分を抽出する

In [ ]:
import cv2
#import numpy as np

cap = cv2.VideoCapture(0)
img_last = None # 前回の画像を記憶する変数
green = (0,255,0)

while True:
    # カメラの画像を読み込む
    _,frame = cap.read()
    # 画像を縮小表示する
    frame = cv2.resize(frame, (500,300))
    #白黒画像に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9,9), 0)
    img_b = cv2.threshold(gray, 100,255, cv2.THRESH_BINARY)[1]
    # 差分を確認する
    if img_last is None:
        img_last = img_b
        continue
    frame_diff = cv2.absdiff(img_last, img_b)
    cnts = cv2.findContours(frame_diff,
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[1]
    # 差分があった点を画面に描く
    for pt in cnts:
        x,y,w,h = cv2.boundingRect(pt)
        if w < 30: continue # 小さな変更点は無視
        cv2.rectangle(frame, (x,y), (x+w, y+h), green, 2)
   
    # 今回のフレームを保存
    img_last = img_b
    # 画面に表示
    cv2.imgshow("DIff Camera", frame)
    cv2.imgshow("diff data", frame_diff)
    if cv2.waitKey(1) == 13: break
cap.release()
cv2.destroyAllWindows()